In [1]:
import pandas as pd

# from data.kbo.statiz import KBOTeams
from data.kbo.naver import NaverGame
from data.kbo.naver import SingleInning

from datetime import datetime

#### Naver Sports (Web crawl)

In [2]:
log = NaverGame("HT", "LT", datetime(2024, 9, 28))
d_ = log.data()

In [3]:
inning_manager = SingleInning(1, "away")
xs, ys = list(), list()
for i, d in enumerate(d_):
    training = inning_manager.insert_log(d)
    if training is not None:
        x, y = training
        xs.append(x)
        ys.append(y)
    else:
        print(f"passing log {i}")
    
    if inning_manager.outs >= 3:
        inning_manager = inning_manager.advance_to_next()  # Change side or inning
        print(f"moved to inning {int(inning_manager.curr_inning)}, {inning_manager.home_away}")


passing log 0
moved to inning 1, home
passing log 5
moved to inning 2, away
passing log 10
moved to inning 2, home
passing log 14
moved to inning 3, away
passing log 20
moved to inning 3, home
passing log 25
moved to inning 4, away
passing log 30
moved to inning 4, home
passing log 34
passing log 39
passing log 43
moved to inning 5, away
passing log 47
moved to inning 5, home
passing log 55
passing log 64
passing log 66
moved to inning 6, away
passing log 69
moved to inning 6, home
passing log 76
passing log 82
moved to inning 7, away
passing log 89
passing log 92
passing log 94
moved to inning 7, home
passing log 96
passing log 99
moved to inning 8, away
passing log 101
moved to inning 8, home
passing log 105
passing log 108
moved to inning 9, away
passing log 110
moved to inning 9, home
passing log 115


In [4]:
pd.concat(xs)

,pitcher,batter,ballCount,type,speed,stance,crossPlateX,crossPlateY,topSz,bottomSz,...,x0,ax,ay,az,strike,ball,out,base1,base2,base3
0,52528,52605,1,투심,142.0,R,-0.078109,0.7083,3.35,1.643,...,1.48108,13.2441,24.7092,-21.2339,0,0,0,0,0,0
0,52528,52667,1,투심,142.0,R,0.538374,0.7083,3.306,1.621,...,1.63494,11.63,28.0619,-21.9436,0,0,0,1,0,0
1,52528,52667,2,체인지업,130.0,R,1.24071,0.7083,3.306,1.621,...,1.76461,14.6725,26.0733,-25.0128,1,0,0,1,0,0
2,52528,52667,3,체인지업,132.0,R,0.105212,0.7083,3.306,1.621,...,1.64999,15.3161,23.2835,-25.6861,2,0,0,1,0,0
3,52528,52667,4,체인지업,129.0,R,0.227236,0.7083,3.306,1.621,...,1.54493,14.0667,26.6,-29.0134,2,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,50662,62802,1,직구,147.0,L,-1.29195,0.7083,3.442,1.689,...,-1.80707,-11.6923,27.441,-23.5202,0,0,2,5,0,0
1,50662,62802,2,직구,144.0,L,-0.319255,0.7083,3.442,1.689,...,-1.59944,-12.276,24.4508,-19.6304,0,1,2,5,0,0
2,50662,62802,3,직구,145.0,L,-0.683633,0.7083,3.442,1.689,...,-1.74205,-7.18508,23.3877,-21.119,1,1,2,5,0,0
3,50662,62802,4,포크,129.0,L,-1.01368,0.7083,3.442,1.689,...,-1.87244,-1.16163,22.1148,-29.7194,2,1,2,5,0,0


In [5]:
pd.concat(ys)

,target
0,1H
0,T
1,S
2,B
3,S
...,...
0,B
1,F
2,S
3,B


In [8]:
total_logs = pd.concat([pd.concat(xs), pd.concat(ys)], axis=1).reset_index(drop=True)

In [9]:
total_logs

,pitcher,batter,ballCount,type,speed,stance,crossPlateX,crossPlateY,topSz,bottomSz,...,ax,ay,az,strike,ball,out,base1,base2,base3,target
0,52528,52605,1,투심,142.0,R,-0.078109,0.7083,3.35,1.643,...,13.2441,24.7092,-21.2339,0,0,0,0,0,0,1H
1,52528,52667,1,투심,142.0,R,0.538374,0.7083,3.306,1.621,...,11.63,28.0619,-21.9436,0,0,0,1,0,0,T
2,52528,52667,2,체인지업,130.0,R,1.24071,0.7083,3.306,1.621,...,14.6725,26.0733,-25.0128,1,0,0,1,0,0,S
3,52528,52667,3,체인지업,132.0,R,0.105212,0.7083,3.306,1.621,...,15.3161,23.2835,-25.6861,2,0,0,1,0,0,B
4,52528,52667,4,체인지업,129.0,R,0.227236,0.7083,3.306,1.621,...,14.0667,26.6,-29.0134,2,1,0,1,0,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,50662,62802,1,직구,147.0,L,-1.29195,0.7083,3.442,1.689,...,-11.6923,27.441,-23.5202,0,0,2,5,0,0,B
330,50662,62802,2,직구,144.0,L,-0.319255,0.7083,3.442,1.689,...,-12.276,24.4508,-19.6304,0,1,2,5,0,0,F
331,50662,62802,3,직구,145.0,L,-0.683633,0.7083,3.442,1.689,...,-7.18508,23.3877,-21.119,1,1,2,5,0,0,S
332,50662,62802,4,포크,129.0,L,-1.01368,0.7083,3.442,1.689,...,-1.16163,22.1148,-29.7194,2,1,2,5,0,0,B
